In [7]:
import itertools
import os
import subprocess
import glob, os, shutil, sys
import numpy as np
import pandas as pd
import xarray as xr
import time

list1 = []
list2 = []
list3 = []
def make_namelist(nml_input,nml_output,station_list,mode='no_spin_up',forcing='FLUXNET-CH4'):
    # open nml file and readlines
    with open(nml_input+'US-Los_FLUXNET-CH4_spin_up_80yr.nml', 'r') as file:
        nml_content = file.readlines()

    path= '/share/home/dq076/data/ME/FLUXNET-CH4/'
    site = f'FLX_{station_list['SITE_ID']}_FLUXNET-CH4_{station_list['YEAR_START']}-{station_list['YEAR_END']}_1-1'
    hh_name = f'FLX_{station_list['SITE_ID']}_FLUXNET-CH4_HH_{station_list['YEAR_START']}-{station_list['YEAR_END']}_1-1.csv'
    name = f"{path}{site}/{hh_name}"
    df = pd.read_csv(name, header=0)

    if df['TIMESTAMP_START'].iloc[0].astype(str)[:4] == str(station_list['YEAR_START']):
        start = str(df['TIMESTAMP_START'].iloc[0])
        end = str(df['TIMESTAMP_START'].iloc[-1])
    else:
        start = str(df['TIMESTAMP_END'].iloc[0])
        end = str(df['TIMESTAMP_END'].iloc[-1])
    start_year = int(start[0:4])
    start_mon = int(start[4:6])
    start_day = int(start[6:8])
    start_second = int(start[8:10])*3600+int(start[10:12])*60
    end_second = int(end[8:10])*3600+int(end[10:12])*60

    generate_nml = True
    if start_mon == 1 and start_day==1 and start_second==0:
        list1.append(station_list['SITE_ID'])
    elif start_year+1<=int(station_list['YEAR_END']):
        list2.append(station_list['SITE_ID'])
        start_year = start_year+1
        start_mon = 1
        start_day = 1
        start_second = 0
    else:
        list3.append(station_list['SITE_ID'])
        generate_nml = False  
    
    if generate_nml:
        for i, line in enumerate(nml_content):
            if 'DEF_CASE_NAME' in line:
                nml_content[i] = f"DEF_CASE_NAME = '{station_list['SITE_ID']}'\n"
            if 'DEF_simulation_time%start_year' in line:
                nml_content[i] = f"DEF_simulation_time%start_year = {start_year}\n"
            if 'DEF_simulation_time%start_month' in line:
                nml_content[i] = f"DEF_simulation_time%start_month = {start_mon}\n"
            if 'DEF_simulation_time%start_day' in line:
                nml_content[i] = f"DEF_simulation_time%start_day = {start_day}\n"
            if 'DEF_simulation_time%start_sec' in line:
                nml_content[i] = f"DEF_simulation_time%start_sec = {start_second}\n"
            if 'DEF_simulation_time%end_year' in line:
                nml_content[i] = f"DEF_simulation_time%end_year = {str(station_list['YEAR_END'])}\n"
            if 'DEF_simulation_time%end_month' in line:
                nml_content[i] = f"DEF_simulation_time%end_month = {int(end[4:6])}\n"
            if 'DEF_simulation_time%end_day' in line:
                nml_content[i] = f"DEF_simulation_time%end_day = {int(end[6:8])}\n"
            if 'DEF_simulation_time%end_sec' in line:
                nml_content[i] = f"DEF_simulation_time%end_sec = {end_second}\n"
            if 'DEF_simulation_time%spinup_year' in line:
                nml_content[i] = f"DEF_simulation_time%spinup_year = {int(station_list['YEAR_START']) - 1}\n"
            if 'SITE_fsitedata' in line:
                nml_content[i] = f"SITE_fsitedata = '{station_list['srfpath']}'\n"
            if 'DEF_dir_output' in line:
                nml_content[i] = f"DEF_dir_output = '/share/home/dq076/data/cases/site/{forcing}/{mode}/'\n"
            if 'DEF_forcing_namelist' in line:
                nml_content[i] = f"DEF_forcing_namelist = '{nml_input}/site/{forcing}/forcing/SINGLE_{station_list['SITE_ID']}.nml'\n"

        new_file_path = f"{nml_output}{station_list['SITE_ID']}.nml"
        with open(new_file_path, 'w') as file:
            file.writelines(nml_content)

if __name__ == '__main__':
    forcing ='FLUXNET-CH4'
    mode='spin_up_80yr'

    stnlist = f"/share/home/dq076/data/ME/FLUXNET-CH4/FLX_AA-Flx_CH4-META_20201112135337801132.csv"
    station_lists = pd.read_csv(stnlist, header=0)
    station_lists = station_lists[station_lists['FLUXNET-CH4_DATA_POLICY'] == 'CCBY4.0'].reset_index(drop=True)
    n = len(station_lists['SITE_ID'])
    nml_input = "/share/home/dq076/mode/ME/CoLM202X_CH4_s/run/"
    nml_output = f"{nml_input}site/{forcing}/{mode}/"
    os.makedirs(nml_output, exist_ok=True)
    for i in range(n):
        station_list = station_lists.iloc[i]
        station_list['srfpath'] = f'/share/home/dq076/data/CoLM_Forcing/PLUMBER2/Srfdata/{station_list['SITE_ID']}_{str(station_list['YEAR_START'])}-{str(station_list['YEAR_END'])}_FLUXNET-CH4_Srf.nc'     
        make_namelist(nml_input,nml_output,station_list,mode,forcing)
    print(list1)
    print(list2)
    print(list3)

/tmp/ipykernel_1999608/1231467906.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_list['srfpath'] = f'/share/home/dq076/data/CoLM_Forcing/PLUMBER2/Srfdata/{station_list['SITE_ID']}_{str(station_list['YEAR_START'])}-{str(station_list['YEAR_END'])}_FLUXNET-CH4_Srf.nc'
/tmp/ipykernel_1999608/1231467906.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_list['srfpath'] = f'/share/home/dq076/data/CoLM_Forcing/PLUMBER2/Srfdata/{station_list['SITE_ID']}_{str(station_list['YEAR_START'])}-{str(station_list['YEAR_END'])}_FLUXNET-CH4_Srf.nc'
/tmp/ipykernel_1999608/1231467906.py:95: SettingWithCopyWarning: 


['AT-Neu', 'BW-Gum', 'BW-Nxr', 'CA-SCB', 'CA-SCC', 'CH-Cha', 'CH-Dav', 'CH-Oe2', 'DE-Dgw', 'DE-Hte', 'DE-SfN', 'DE-Zrk', 'FI-Hyy', 'FI-Lom', 'FI-Si2', 'FI-Sii', 'FR-LGt', 'HK-MPM', 'IT-BCi', 'IT-Cas', 'JP-BBY', 'JP-Mse', 'JP-SwL', 'KR-CRK', 'MY-MLM', 'NL-Hor', 'NZ-Kop', 'PH-RiF', 'RU-Ch2', 'RU-Che', 'RU-Cok', 'RU-Fy2', 'SE-Deg', 'UK-LBT', 'US-A03', 'US-A10', 'US-Atq', 'US-Beo', 'US-Bes', 'US-Bi1', 'US-Bi2', 'US-BZB', 'US-BZF', 'US-BZS', 'US-CRT', 'US-DPW', 'US-EDN', 'US-EML', 'US-Ho1', 'US-HRA', 'US-HRC', 'US-Ivo', 'US-LA1', 'US-Los', 'US-MRM', 'US-Myb', 'US-NC4', 'US-NGB', 'US-ORv', 'US-OWC', 'US-Pfa', 'US-Snd', 'US-Sne', 'US-Srr', 'US-StJ', 'US-Tw1', 'US-Tw3', 'US-Tw4', 'US-Tw5', 'US-Twt', 'US-Uaf', 'US-WPT']
['BR-Npw', 'CN-Hgu', 'ID-Pag', 'US-ICs', 'US-LA2', 'US-MAC', 'US-NGC']
[]
